In [123]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torchinfo import summary
from torchvision.models import vgg19
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import tensorflow as tf
import torch.nn as nn
from math import sqrt
import pandas as pd
import numpy as np
import random
import torch
import os
# import imshow

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')


Running on the CPU


In [119]:
# Data = np.load("../data/Dataset.npy",allow_pickle=True)
X = np.load("../data/Images.npy",allow_pickle=True)
y= np.load("../data/Labels.npy",allow_pickle=True)
X.shape , y.shape , X.dtype , y.dtype

((130, 224, 224, 3), (130, 19), dtype('int32'), dtype('int32'))

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape    

((104, 224, 224, 3), (26, 224, 224, 3), (104, 19), (26, 19))

In [121]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [96]:
model = vgg19(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [01:04<00:00, 8.95MB/s] 


In [122]:
summary(model, input_size=(1, 3, 224, 224), verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

In [124]:
# training the model for face recognition

n_epochs = 10
batch_size = 5
lr = 0.3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

train_losses = []
test_losses = []
train_acc = []
test_acc = []

for epoch in range(n_epochs):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy: {}/{} ({:.0f}%)'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), correct, len(data),
                100. * correct / len(data)))
            

    # train_losses.append(loss.item())
    # train_acc.append(100. * correct / len(train_loader.dataset))
    # model.eval()
    # correct = 0
    # with torch.no_grad():
    #     for batch_idx, (data, target) in enumerate(test_loader):
    #         data = data.to(device)
    #         target = target.to(device)
    #         output = model(data)
    #         loss = loss_fn(output, target)
    #         pred = output.argmax(dim=1, keepdim=True)
    #         correct += pred.eq(target.view_as(pred)).sum().item()
    # loss = loss.item()
    # test_losses.append(loss)
    # test_acc.append(100. * correct / len(test_loader.dataset))
    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     loss, correct, len(test_loader.dataset),
    #     100. * correct / len(test_loader.dataset)))




KeyboardInterrupt

